In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

import sys
sys.path.append('../')

from wavy import panel
from wavy import side
from wavy import block
from wavy import nn
from wavy import plot

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
df.head(5)

LNC                                                    MAS  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2005-12-21  38.127885  39.324626  38.120589  39.207870  2190100.0  18.991695   
2005-12-22  39.185984  39.324630  38.879500  39.040039  1095500.0  19.085703   
2005-12-23  39.061931  39.397604  38.959771  39.324631   588800.0  18.954090   
2005-12-27  39.514374  39.514374  38.981676  39.237080   932700.0  18.929008   
2005-12-28  39.455973  39.696782  39.244353  39.310028   427200.0  18.853798   

                                                        ...       CSX  \
                 High        Low      Close     Volume  ...      Open   
Date                                                    ...             
2005-12-21  19.154659  18.903944  19.085712  1384200.0  ...  6.245725   
2005-12-22  19.298810  18.910202  18.929007  1576900.0  ...  6.381336   
2005-12-23  18.954090  18.684571  18.816196  2090500.0  ...  6.297691   
2005-12-27  18.991687  18.803650  18.847525  1609000.0  ...  6.520750   
2005-12-28  18.935281  18.747245  18.803656  2026300.0  ...  6.419358   

                                                            CI             \
                High       Low     Close     Volume       Open       High   
Date                                                                        
2005-12-21  6.399080  6.245725  6.362326  8358000.0  36.945102  37.462575   
2005-12-22  6.420626  6.361058  6.410487  5344800.0  37.409841  37.630677   
2005-12-23  6.520752  6.297691  6.468790  3630000.0  37.376876  37.432909   
2005-12-27  6.527087  6.369931  6.400348  4905000.0  37.195584  37.455973   
2005-12-28  6.459914  6.363592  6.419358  3578400.0  37.241735  37.449383   

                                             
                  Low      Close     Volume  
Date                                         
2005-12-21  36.895658  37.370289  2805900.0  
2005-12-22  37.073647  37.327442  2296800.0  
2005-12-23  36.905546  37.261517  1686000.0  
2005-12-27  36.961570  37.116482  1917900.0  
2005-12-28  37.109892  37.251621  1248900.0  

[5 rows x 2085 columns]

In [4]:
timepanel = panel.from_data(df, lookback = 5, horizon = 2, gap = 0, assets = ['AAPL', 'MSFT'], channels= ['High', 'Low'])

In [5]:
timepanel.x[0]

AAPL                 MSFT           
                High       Low       High        Low
Date                                                
2005-12-21  2.249430  2.216732  19.606268  19.460550
2005-12-22  2.276323  2.249125  19.511545  19.249254
2005-12-23  2.269293  2.239957  19.431410  19.263835
2005-12-27  2.297408  2.259821  19.562552  19.271118
2005-12-28  2.284574  2.240569  19.424128  19.198267

In [6]:
timepanel.y[0]

AAPL                 MSFT           
                High       Low       High        Low
Date                                                
2005-12-29  2.255848  2.182507  19.307547  19.132687
2005-12-30  2.213372  2.149504  19.169114  19.016111

In [7]:
timepanel.set_training_split()

In [8]:
timepanel.x.first

AAPL                 MSFT           
                High       Low       High        Low
Date                                                
2005-12-21  2.249430  2.216732  19.606268  19.460550
2005-12-22  2.276323  2.249125  19.511545  19.249254
2005-12-23  2.269293  2.239957  19.431410  19.263835
2005-12-27  2.297408  2.259821  19.562552  19.271118
2005-12-28  2.284574  2.240569  19.424128  19.198267

In [9]:
timepanel.shape

100%|██████████| 3741/3741 [00:00<00:00, 7255.40it/s]


,size,assets,timesteps,channels
x,3741,2,5,2
y,3741,2,2,2


# Baseline

In [10]:
baseline = nn.Baseline(timepanel, metrics=['SE', 'MSE', 'RMSE'])

In [11]:
predicted = baseline.predict()

In [12]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [13]:
predicted[2]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-22  45.015131  44.021305  125.735111  123.931721
2019-05-23  45.906399  45.100595  125.675984  124.739798

In [14]:
predicted_val = baseline.predict_val()

In [15]:
timepanel.val.y[0]

AAPL                  MSFT           
                 High        Low       High        Low
Date                                                  
2016-09-14  26.409318  25.374254  52.876488  52.316255
2016-09-15  27.040171  26.516797  53.548759  52.269566

In [16]:
predicted_val[2]

AAPL                  MSFT           
                 High        Low       High        Low
Date                                                  
2016-09-16  26.409318  25.374254  52.876488  52.316255
2016-09-19  27.040171  26.516797  53.548759  52.269566

In [17]:
baseline.evaluate('test')

100%|██████████| 374/374 [00:00<00:00, 7564.70it/s]


[40347.44818240013, 13.48510968663106, 3.672207740124605]

In [18]:
# m = tf.keras.metrics.MeanSquaredError()
# m.update_state(timepanel.test.y.tensor3d, predicted.tensor3d)
# m.result().numpy()

# DenseModel

In [19]:
densemodel = nn.DenseModel(timepanel, model_type="regression")

100%|██████████| 374/374 [00:00<00:00, 156741.58it/s]
2021-12-25 15:48:50.707043: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
densemodel.fit()

85/85 [==============================] - 1s 2ms/step - loss: 244.9110 - MAE: 10.6662 - val_loss: 69.0523 - val_MAE: 6.0856


In [21]:
densemodel.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 20)                0         
                                                                 
 dense (Dense)               (None, 32)                672       
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
 reshape (Reshape)           (None, 2, 4)              0         
                                                                 
Total params: 936
Trainable params: 936
Non-trainable params: 0
_________________________________________________________________


In [22]:
predicted = densemodel.predict()

In [23]:
predicted[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  61.537758  42.401218  119.656433   95.543556
2019-05-21  56.708359  49.321712  101.235733  121.275902

In [24]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [25]:
densemodel.evaluate('test')

12/12 [==============================] - 0s 1ms/step - loss: 306.5314 - MAE: 13.2663


[306.53143310546875, 13.266328811645508]

# ConvModel

In [26]:
convmodel = nn.ConvModel(timepanel, model_type="regression")

100%|██████████| 374/374 [00:00<00:00, 174996.62it/s]
/Users/ibiscp/GitHub/wavy/notebooks/../wavy/nn.py:249: UserWarning: Kernel size is not a divisor of lookback.
  warnings.warn("Kernel size is not a divisor of lookback.")


In [27]:
convmodel.fit()

85/85 [==============================] - 1s 2ms/step - loss: 88.9809 - MAE: 5.6444 - val_loss: 5.6301 - val_MAE: 1.6554


In [28]:
convmodel.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 32)             416       
                                                                 
 flatten_1 (Flatten)         (None, 96)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                3104      
                                                                 
 dense_3 (Dense)             (None, 8)                 264       
                                                                 
 reshape_1 (Reshape)         (None, 2, 4)              0         
                                                                 
Total params: 3,784
Trainable params: 3,784
Non-trainable params: 0
_________________________________________________________________


In [29]:
predicted = convmodel.predict()

In [30]:
predicted[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  47.930233  46.511143  119.839737  126.021828
2019-05-21  54.090282  50.642174  119.397881  122.789276

In [31]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [32]:
convmodel.evaluate()

12/12 [==============================] - 0s 1ms/step - loss: 35.0828 - MAE: 4.4490


[35.08277130126953, 4.448971748352051]

# LinearRegression

In [33]:
linearRegression = nn.LinearRegression(timepanel)

100%|██████████| 374/374 [00:00<00:00, 158547.57it/s]


In [34]:
linearRegression.fit()

85/85 [==============================] - 0s 2ms/step - loss: 607.0486 - MAE: 17.1266 - val_loss: 1625.9465 - val_MAE: 25.4193


In [35]:
predicted = linearRegression.predict()

In [36]:
predicted[0]

AAPL                   MSFT           
                 High        Low        High        Low
Date                                                   
2019-05-20  59.529209  23.815462  -18.610277  72.183418
2019-05-21  44.009869  40.264786  125.245056  79.433899

In [37]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [38]:
linearRegression.evaluate()

12/12 [==============================] - 0s 2ms/step - loss: 6275.5259 - MAE: 50.6630


[6275.52587890625, 50.663002014160156]

# Teste SeparateAssetModel

In [39]:
timepanel.val.x[0]

AAPL                  MSFT           
                 High        Low       High        Low
Date                                                  
2016-09-07  25.411634  25.016767  54.006285  53.604785
2016-09-08  25.063503  24.589196  53.959598  53.390029
2016-09-09  24.701344  24.096193  53.707498  52.484325
2016-09-12  24.701349  23.956008  53.418049  51.924100
2016-09-13  25.418647  25.056491  52.895164  52.334931

In [40]:
test_splits = timepanel.val.x._split_assets()

100%|██████████| 673/673 [00:04<00:00, 158.54it/s]


In [41]:
x_train = [x.tensor3d for x in test_splits]

100%|██████████| 673/673 [00:00<00:00, 7100.87it/s]


In [42]:
# pd.DataFrame(x_train)

ValueError: Must pass 2-d input. shape=(2, 673, 5, 2)

In [43]:
test_splits[0].tensor3d[0]

100%|██████████| 673/673 [00:00<00:00, 168503.26it/s]


array([[25.4116342 , 25.01676731],
       [25.06350266, 24.58919594],
       [24.70134446, 24.09619331],
       [24.70134857, 23.95600842],
       [25.41864747, 25.05649119]])

In [44]:
test_splits[0].tensor4d[0]

100%|██████████| 673/673 [00:00<00:00, 176964.87it/s]


array([[[25.4116342 , 25.01676731],
        [25.06350266, 24.58919594],
        [24.70134446, 24.09619331],
        [24.70134857, 23.95600842],
        [25.41864747, 25.05649119]]])

In [45]:
pd.DataFrame(timepanel.val.x.tensor4d)

100%|██████████| 673/673 [00:00<00:00, 160923.93it/s]


ValueError: Must pass 2-d input. shape=(673, 2, 5, 2)

# SeparateAssetModel

In [46]:
separateAssetModel = nn.SeparateAssetModel(timepanel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

100%|██████████| 374/374 [00:00<00:00, 134719.14it/s]
/Users/ibiscp/GitHub/wavy/notebooks/../wavy/nn.py:249: UserWarning: Kernel size is not a divisor of lookback.
  warnings.warn("Kernel size is not a divisor of lookback.")


In [47]:
separateAssetModel.fit()

85/85 [==============================] - 1s 4ms/step - loss: 148.0462 - MAE: 8.0185 - val_loss: 14.8862 - val_MAE: 2.6888


In [48]:
separateAssetModel.model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 2, 3, 32)          224       
                                                                 
 flatten_3 (Flatten)         (None, 192)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                6176      
                                                                 
 dense_7 (Dense)             (None, 8)                 264       
                                                                 
 reshape_3 (Reshape)         (None, 2, 2, 2)           0         
                                                                 
Total params: 6,664
Trainable params: 6,664
Non-trainable params: 0
_________________________________________________________________


In [49]:
predicted = separateAssetModel.predict()

In [50]:
predicted[0]

AAPL                    MSFT            
                  High         Low        High         Low
Date                                                      
2019-05-20   54.686153   50.517326   51.474285   50.571041
2019-05-21  113.824753  114.460777  114.389969  107.281631

In [51]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [52]:
separateAssetModel.evaluate()

12/12 [==============================] - 0s 962us/step - loss: 84.7599 - MAE: 7.4542


[84.75994873046875, 7.454155921936035]

# SeparateAssetConvModel

In [53]:
separateAssetConvModel = nn.SeparateAssetConvModel(timepanel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

100%|██████████| 374/374 [00:00<00:00, 180431.30it/s]
/Users/ibiscp/GitHub/wavy/notebooks/../wavy/nn.py:249: UserWarning: Kernel size is not a divisor of lookback.
  warnings.warn("Kernel size is not a divisor of lookback.")


In [54]:
separateAssetConvModel.fit()

85/85 [==============================] - 1s 2ms/step - loss: 188.1015 - MAE: 9.4851 - val_loss: 20.3925 - val_MAE: 3.2924


In [55]:
separateAssetConvModel.model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 2, 3, 32)          224       
                                                                 
 flatten_4 (Flatten)         (None, 192)               0         
                                                                 
 dense_8 (Dense)             (None, 32)                6176      
                                                                 
 dense_9 (Dense)             (None, 8)                 264       
                                                                 
 reshape_4 (Reshape)         (None, 2, 2, 2)           0         
                                                                 
Total params: 6,664
Trainable params: 6,664
Non-trainable params: 0
_________________________________________________________________


In [56]:
predicted = separateAssetConvModel.predict()

In [57]:
predicted[0]

AAPL                    MSFT            
                  High         Low        High         Low
Date                                                      
2019-05-20   55.095802   58.526108   61.571705   54.702652
2019-05-21  116.096306  112.927116  113.052429  117.125519

In [58]:
timepanel.test.y[0]

AAPL                   MSFT            
                 High        Low        High         Low
Date                                                    
2019-05-20  45.015131  44.021305  125.735111  123.931721
2019-05-21  45.906399  45.100595  125.675984  124.739798

In [59]:
separateAssetConvModel.evaluate()

12/12 [==============================] - 0s 1ms/step - loss: 103.1822 - MAE: 8.9774


[103.18221282958984, 8.977449417114258]

# Test Docstrings

In [60]:
densemodel = nn.DenseModel()

SyntaxError: invalid syntax (823336136.py, line 1)